In [1]:
import numpy as np
import matplotlib.pyplot as plt

project = 'newfault'
traindate = '2024-10-01'
testdate = '2024-10-02'
traindate_path = '/home/sdybing/gnss-picker/cnn_models_outputs/' + project + '_fq_train/models/traindate_' + traindate + '/'
test_outputs_path = traindate_path + 'data/'
figure_save_dir = traindate_path + 'figures/'

fqtest_data = np.load(test_outputs_path + testdate + '_fqtest_norm_data.npy')
fqtest_metadata = np.load(test_outputs_path + testdate + '_fqtest_metadata.npy')
fqtest_target = np.load(test_outputs_path + testdate + '_fqtest_target.npy')
fqtest_predictions = np.load(test_outputs_path + testdate + '_fqtest_predictions.npy')




In [6]:
num_fqtest = len(fqtest_predictions)
best_thresh = 0.025 # From code 2

zeros = np.zeros((fqtest_predictions.shape[0],1))
analysis_array = np.c_[fqtest_metadata, zeros] # Adds a column of zeros to the metadata array to initialize

In [7]:
for i in range(len(fqtest_metadata)):
    
    if fqtest_metadata[i][0] == 'nan': # This row is NOT an earthquake
        
        # Make binary prediction and target
        
        p = np.where(fqtest_predictions[i] >= best_thresh)[0]
        if len(p) == 0:
            pred_binary = 0
        elif len(p) > 0:
            pred_binary = 1
        
        t = np.where(fqtest_target[i] > 0)[0]
        if len(t) == 0:
            targ_binary = 0
        elif len(t) > 0:
            targ_binary = 1
        
        pred = pred_binary
        targ = targ_binary
        
        # Figure out what the result of the prediction is
        
        if pred == targ:
            if pred == 1 and targ == 1:
                result = 'true pos'
            elif pred == 0 and targ == 0:
                result = 'true neg'
            
        elif pred != targ:
            if pred == 1 and targ == 0:
                result = 'false pos'
            elif pred == 0 and targ == 1:
                result = 'false neg'
        
        analysis_array[i][3] = result
    
    else: # This row IS an earthquake
        
        rupt_num = fqtest_metadata[i][0]
        station = fqtest_metadata[i][1]
        mag = fqtest_metadata[i][2]
        
        p = np.where(fqtest_predictions[i] >= best_thresh)[0]
        if len(p) == 0:
            pred_binary = 0
        elif len(p) > 0:
            pred_binary = 1
        
        t = np.where(fqtest_target[i] > 0)[0]
        if len(t) == 0:
            targ_binary = 0
        elif len(t) > 0:
            targ_binary = 1
        
        pred = pred_binary
        targ = targ_binary
        
        if pred == targ:
            if pred == 1 and targ == 1:
                result = 'true pos'
            elif pred == 0 and targ == 0:
                result = 'true neg'
            
        elif pred != targ:            
            if pred == 1 and targ == 0:
                result = 'false pos'
            elif pred == 0 and targ == 1:
                result = 'false neg'
        
        analysis_array[i][3] = result

In [8]:
print(analysis_array)
print(analysis_array.shape)
            
np.save(test_outputs_path + 'fakequakes_testing/fqtest_metadata_withresults_0025.npy', analysis_array)  

[['newfault.001132' 'NHRG' '6.3333' 'true pos']
 ['newfault.000179' 'P462' '7.0808' 'true pos']
 ['nan' 'nan' 'nan' 'false pos']
 ...
 ['newfault.002773' 'BKAP' '4.7904' 'true pos']
 ['newfault.000388' 'P091' '7.0220' 'true pos']
 ['newfault.003291' 'P289' '4.3230' 'true pos']]
(91740, 4)
